# Each County Graphed by State

#### Import Libraries

In [1]:
import pandas as pd
from datetime import datetime
import random
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.models import LinearAxis, Range1d, HoverTool, CustomJS
from bokeh.models.widgets import CheckboxGroup
from bokeh.io import output_file, show
from bokeh.resources import CDN
from bokeh.embed import file_html

#### Read in .csv file and store in DataFrame

In [2]:
file = "us-counties.csv"
df = pd.read_csv(file)
df = df.drop(columns='fips')

#### Get colors from colors.txt

In [3]:
colorList = []
colorlist = open("colors.txt","r")
for color in colorlist:
    colorList.append(color)
colorlist.close()

#### Graph each county by state

In [4]:
for num,state in enumerate(sorted(df['state'].unique())):
    
    plot_Cases = []
    plot_Deaths = []
    
    stateDF = df[df['state'] == state]
    
    #Figure graph
    plotWidth = 1000
    plotHeight = 500
    startRange = 0
    endRange = int(stateDF['cases'].max()*.8)
    graphs = figure(title = state, x_axis_label='DATE', y_axis_label='CONFIRMED CASES OF COVID-19', x_axis_type='datetime',y_range=(startRange,endRange),tools='pan,box_zoom,zoom_in,zoom_out,reset,save',plot_width=plotWidth, plot_height=plotHeight)    
    graphs.extra_y_ranges = {"plot": Range1d(start=startRange, end=endRange)}
    graphs.add_layout(LinearAxis(y_range_name="plot", axis_label='DECEASED FROM COVID-19'), 'right')
    graphs.left[0].formatter.use_scientific = False
    graphs.right[0].formatter.use_scientific = False
    graphs.add_tools(HoverTool(tooltips=[("", "$name"),("","$x{%F}: $y{int}")],formatters = {"$x": "datetime"},mode='mouse',toggleable=False)) 


    for county in sorted(stateDF['county'].unique()):
        countyDF = stateDF[stateDF['county'] == county]
                
        dates = pd.to_datetime(countyDF['date'], format='%Y-%m-%d')
        cases = countyDF['cases']
        deaths = countyDF['deaths']

        color = random.choice(colorList)
        color = color.strip('\n')

        plot_Cases.append(graphs.line(dates,cases,line_width=2,line_color=color,name=county,legend_label="Confirmed"))
        plot_Deaths.append(graphs.line(dates,deaths,line_width=2,line_dash='dashed',line_color=color,name=county, y_range_name="plot",legend_label="Deceased"))

        
    #Finish figuring graph
    graphs.legend.location = 'top_left'
    checkboxes = CheckboxGroup(labels=sorted(stateDF['county'].unique()))
    
    #Linking with JavaScript
    callback = CustomJS(code="""
                                var i;
                                for (i = 0; i < %i; i++)
                                {
                                    if (cb_obj.active.includes(i))
                                    {
                                        lineC[i].visible = true;
                                        lineD[i].visible = true;
                                    }
                                    else
                                    {
                                        lineC[i].visible = false;
                                        lineD[i].visible = false;
                                    }
                                }
                                """ % len(stateDF['county'].unique()),
                        args={'lineC': plot_Cases, 'lineD': plot_Deaths})

    checkboxes.js_on_click(callback)


    
    #Show Graph
    output_file(f"{state}CountiesGraph.html")
    show(row(graphs,checkboxes))

    #Store as html
    # html = file_html(row(graphs,checkboxes), CDN, "US States")
    # fa = open("stateGraphs.txt","w")
    # fa.write(html)
    # fa.close()
    
    #Break after 5
    if num == 4:
        break